In [16]:
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import re

# MDL Website

In [17]:
# URL to scrape
url = 'https://mydramalist.com/shows/top?page=1'
# Send a GET request to the URL
response = requests.get(url)
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [25]:
from bsoup_scrape import bsoup_scrape_data

ImportError: cannot import name 'bsoup_scrape_data' from 'bsoup_scrape' (/Users/camilla/Desktop/StatClasses/stat386/termProject/termProject/code/bsoup_scrape.py)

In [20]:
years = []
episodes = []
countries = []
titles = []
scores = []
ranks = []
genres = []
networks = []
watchers = []


bsoup_scrape_data(titles, years, episodes, countries, scores, ranks, soup)

ImportError: cannot import name 'bsoup_scrape_data' from 'bsoup_scrape' (/Users/camilla/Desktop/StatClasses/stat386/termProject/termProject/code/bsoup_scrape.py)

In [3]:
years = []
episodes = []
countries = []
titles = []
scores = []
ranks = []
genres = []
networks = []
watchers = []

# SHOW TITLES
show_titles = soup.find_all('h6', class_='text-primary')
for title in show_titles:
    title = title.text.strip()
    titles.append(title)

# COUNTRY / YEAR / EPISODES
show_info = soup.find_all('span', class_='text-muted')
for info in show_info:
    # get the info
    info = info.text.strip()

    # extract the year
    year = info.split(',')[0][-4:]
    year = int(year)
    years.append(year)

    # extract the episode
    episode = info.split(',')[1]
    numbers = re.findall(r'\d+', episode)
    ep_length= int(numbers[0])
    episodes.append(ep_length)

    country = info.split(' ')[0] 
    countries.append(country)

# SHOW SCORES
show_scores = soup.find_all('span', class_='score')
for score in show_scores:
    score = score.text.strip()
    scores.append(score)

# SHOW RANKS
show_rank = soup.find_all('div', class_='ranking pull-right')
for rank in show_rank:
    rank = rank.text.strip()
    rank = int(rank.split("#")[1])
    ranks.append(rank)


In [11]:
my_drama_list = pd.DataFrame({
    "title": titles,
    "year": years,
    "episodes": episodes,
    "viewer_score": scores,
    "rank": ranks
})

mdl = pd.DataFrame(my_drama_list)
mdl.to_csv("mdl.csv", index=False)
mdl

,title,year,episodes,viewer_score,rank
0,Twinkling Watermelon,2023,16,9.2,1
1,Move to Heaven,2021,10,9.1,2
2,Moving,2023,20,9.1,3
3,Weak Hero Class 1,2022,8,9.1,4
4,Hospital Playlist Season 2,2021,12,9.1,5
5,Nirvana in Fire,2015,54,9.1,6
6,Flower of Evil,2020,16,9.1,7
7,Alchemy of Souls,2022,20,9.1,8
8,Hospital Playlist,2020,12,9.1,9
9,Reset,2022,15,9.1,10


In [5]:
# Find the elements containing the content ratings
ratings = soup.find_all('span', class_='text-primary-dark')

# Extract and print the ratings
for rating in ratings:
    print(rating.text.strip())


In [6]:
# Find the elements containing the genres
genres = soup.find_all('span', class_='genre')

# Extract and print the genres
for genre in genres:
    print(genre.text.strip())

In [7]:
# Find the elements containing the tags
tags = soup.find_all('span', class_='tag')

# Extract and print the tags
for tag in tags:
    print(tag.text.strip())

NEW
NEW
Wanted
NEW
NEW
NEW
Wanted


In [8]:
# Find the elements containing the main cast
cast_elements = soup.find_all('div', class_='text-primary')

# Extract and print the main cast
for cast_element in cast_elements:
    cast = cast_element.text.strip()
    print(cast)

In [9]:
# Find the element containing the synopsis
synopsis_elements = soup.find_all('div', class_='synopsis')

# Extract and print the synopses
for synopsis_element in synopsis_elements:
    synopsis = synopsis_element.text.strip()
    print(synopsis)